# Project 5 - Machine Learning Model in Python

Artificial intelligence has been with us for a long time, but it has never developed at such a dizzying pace as today. Its models accompany us at every step. Starting with assistants in phones and online stores who recommend applications or products to complex language models such as ChatGPT or Bard.

In this project, I would like to use a popular set of tools in the most popular programming language, Python, in order not only to better understand the relationship between the data used in previous projects, but also to use them to build a model that will allow predicting the grades obtained by gymnastics competitors!

Tools used:

- Jupyter notebook

- Python 3.11.2

At the very beginning, we will prepare the environment in which we will work!

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline